In [140]:
import requests
import requests_cache
import json
import pandas as pd
import time
from tqdm._tqdm_notebook import tqdm_notebook
requests_cache.install_cache()

In [113]:
r = requests.get("https://api.thingspeak.com/channels/870146/feeds.json?results={}".format("10"))
r.status_code

200

In [21]:
def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)   

In [22]:
jprint(r.json())
len(r.json()["feeds"])

{
    "channel": {
        "created_at": "2019-09-23T13:03:40Z",
        "description": "AirQo unit with dual pms5003 sensor Cocis",
        "field1": "Sensor1 PM2.5_CF_1_ug/m3",
        "field2": "Sensor1 PM10_CF_1_ug/m3",
        "field3": "Sensor2PM2.5_CF_1_ug/m3",
        "field4": "Sensor2 PM10_CF_1_ug/m3",
        "field5": "Latitude",
        "field6": "Longitude",
        "field7": "Battery Voltage",
        "field8": "GpsData",
        "id": 870146,
        "last_entry_id": 110829,
        "latitude": "0.0",
        "longitude": "0.0",
        "name": "AIRQO-62 UNIT ACTIVE",
        "updated_at": "2019-09-30T10:14:21Z"
    },
    "feeds": [
        {
            "created_at": "2020-02-11T10:20:19Z",
            "entry_id": 110820,
            "field1": "   43.95",
            "field2": "   54.28",
            "field3": "   47.87",
            "field4": "   60.03",
            "field5": "0.332360",
            "field6": "32.569843",
            "field7": "    4.21",
           

10

In [40]:
feed = r.json()
feed_df = pd.DataFrame(feed['feeds'])
feed_df.head()

,created_at,entry_id,field1,field2,field3,field4,field5,field6,field7,field8
0,2020-02-11T10:20:19Z,110820,43.95,54.28,47.87,60.03,0.332360,32.569843,4.21,"0.332360,32.569843,1223.70,0.00,11.00,80.00,0...."
1,2020-02-11T10:21:40Z,110821,45.43,55.53,48.20,61.27,0.332300,32.569836,4.21,"0.332300,32.569836,1230.60,0.00,11.00,80.00,0...."
2,2020-02-11T10:22:54Z,110822,43.23,53.45,47.93,58.53,0.332298,32.569874,4.21,"0.332298,32.569874,1216.80,0.00,11.00,90.00,0...."
3,2020-02-11T10:24:23Z,110823,44.87,54.68,49.90,62.15,0.332335,32.569874,4.21,"0.332335,32.569874,1211.30,0.00,10.00,100.00,0..."
4,2020-02-11T10:25:45Z,110824,46.00,56.03,48.70,60.60,0.332290,32.569847,4.21,"0.332290,32.569847,1201.70,0.00,11.00,90.00,0...."


In [56]:
dates = feed_df['created_at'][0]
new_end = dates[:-1].replace("T","%20")
new_end

'2020-02-11%2010:20:19'

In [58]:
r2 = requests.get(
    "https://api.thingspeak.com/channels/870146/feeds.json?results={}&end={}".format("10", new_end))

In [72]:
feed = r2.json()
feed_df = pd.DataFrame(feed['feeds'])
feed_df.head()
r2.json()['feeds'][0]['created_at']

,created_at,entry_id,field1,field2,field3,field4,field5,field6,field7,field8
0,2020-02-11T10:07:50Z,110811,45.65,55.48,48.38,60.33,0.332277,32.569874,4.21,"0.332277,32.569874,1218.60,0.00,14.00,80.00,0...."
1,2020-02-11T10:09:11Z,110812,45.48,56.15,48.67,60.28,0.332313,32.569897,4.21,"0.332313,32.569897,1223.00,0.00,12.00,90.00,0...."
2,2020-02-11T10:10:33Z,110813,45.62,56.08,49.45,60.77,0.332327,32.569904,4.21,"0.332327,32.569904,1228.00,0.00,13.00,90.00,0...."
3,2020-02-11T10:11:54Z,110814,45.40,55.43,50.05,61.38,0.332343,32.569897,4.21,"0.332343,32.569897,1232.60,0.00,12.00,80.00,0...."
4,2020-02-11T10:13:23Z,110815,43.70,53.28,47.95,59.40,0.332318,32.569866,4.21,"0.332318,32.569866,1231.00,0.00,13.00,80.00,0...."


'2020-02-11T10:07:50Z'

In [147]:
def get_feeds():
    new_end = ""
    responses = []

    while True:  
        r = requests.get(
        "https://api.thingspeak.com/channels/870146/feeds.json?results={}&end={}".format("8000", new_end))
        responses.append(r)

        if len(r.json()["feeds"]) < 8000:
            return responses
        else:
            new_end = r.json()['feeds'][0]['created_at'][:-1].replace("T","%20")
            time.sleep(0.25)    

In [148]:
def feed_data():
    
    responses = get_feeds()
    frames = [pd.DataFrame(response.json()['feeds'][:-1]) for response in responses[::-1]]
    feeds = pd.concat(frames)
    
    return feeds

In [145]:
feeds = feed_data()
feeds.info

<bound method DataFrame.info of               created_at  entry_id    field1    field2    field3    field4  \
0   2020-02-10T13:01:45Z    109896     27.73     30.18     32.07     37.22   
1   2020-02-10T13:03:06Z    109897     26.37     28.00     29.52     32.65   
2   2020-02-10T13:04:27Z    109898     26.77     28.92     30.28     32.83   
3   2020-02-10T13:05:55Z    109899     29.35     32.08     32.02     36.02   
4   2020-02-10T13:07:16Z    109900     29.15     31.03     29.25     32.57   
..                   ...       ...       ...       ...       ...       ...   
94  2020-02-11T11:47:41Z    110881     43.43     53.08     46.15     56.32   
95  2020-02-11T11:49:03Z    110882     41.60     50.50     44.40     54.03   
96  2020-02-11T11:50:25Z    110883     42.80     51.58     45.17     55.30   
97  2020-02-11T11:51:46Z    110884     41.95     49.83     44.58     54.10   
98  2020-02-11T11:53:15Z    110885     39.92     47.43     43.73     53.78   

      field5     field6    fiel

In [146]:
feeds.head(20)

,created_at,entry_id,field1,field2,field3,field4,field5,field6,field7,field8
0,2020-02-10T13:01:45Z,109896,27.73,30.18,32.07,37.22,0.332365,32.569889,4.21,"0.332365,32.569889,1230.30,0.00,11.00,80.00,0...."
1,2020-02-10T13:03:06Z,109897,26.37,28.00,29.52,32.65,0.332357,32.569881,4.21,"0.332357,32.569881,1228.90,0.00,11.00,80.00,0...."
2,2020-02-10T13:04:27Z,109898,26.77,28.92,30.28,32.83,0.332328,32.569893,4.21,"0.332328,32.569893,1222.90,0.00,10.00,100.00,0..."
3,2020-02-10T13:05:55Z,109899,29.35,32.08,32.02,36.02,0.332368,32.569881,4.21,"0.332368,32.569881,1216.80,0.00,11.00,80.00,0...."
4,2020-02-10T13:07:16Z,109900,29.15,31.03,29.25,32.57,0.332323,32.569889,4.21,"0.332323,32.569889,1219.70,0.00,11.00,80.00,0...."
5,2020-02-10T13:08:36Z,109901,27.47,29.35,29.80,32.88,0.332348,32.569908,4.21,"0.332348,32.569908,1222.30,0.00,11.00,80.00,0...."
6,2020-02-10T13:09:56Z,109902,29.17,31.80,31.70,33.95,0.332348,32.569916,4.21,"0.332348,32.569916,1219.60,0.00,10.00,90.00,0...."
7,2020-02-10T13:11:25Z,109903,27.65,30.20,31.17,33.58,0.332350,32.569920,4.21,"0.332350,32.569920,1219.60,0.00,10.00,90.00,0...."
8,2020-02-10T13:12:45Z,109904,27.27,30.02,30.88,32.45,0.332340,32.569923,4.21,"0.332340,32.569923,1221.00,0.00,10.00,90.00,0...."
9,2020-02-10T13:14:07Z,109905,27.48,30.45,31.40,34.75,0.332350,32.569931,4.21,"0.332350,32.569931,1219.80,0.00,8.00,100.00,0...."


# 